런타임 유형변경 >> CPU

In [0]:
# 구글 드라이브와 Colab 연동
from google.colab import drive
drive.mount('/content/drive') # 출력되는 URL에 접속하여 verification code 복사 및 붙여넣기

In [0]:
# 작업할 공간 설정
import os
os.getcwd() # 현재 path 확인

In [0]:
os.chdir('/content/drive/My Drive/PyConKorea2019-Tutorials/LRP') # 작업할 path를 google drive로 이동
os.getcwd()

In [0]:
!pip install innvestigate

In [0]:
import innvestigate
import innvestigate.utils
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from keras.applications import vgg16, imagenet_utils

In [0]:
def load_image(path, size):
    ret = PIL.Image.open(path)
    print(ret)
    ret = ret.resize((size, size))
    print(ret)
    ret = np.asarray(ret, dtype=np.uint8).astype(np.float32)
    print(ret.shape)
    if ret.ndim == 2:
        ret.resize((size, size, 1))
        ret = np.repeat(ret, 3, axis=-1)
    return ret

In [0]:
image_path = "husky.jpg"

# Load data and preprocess
img = Image.open(image_path)
img = img.resize((224, 224),resample=PIL.Image.NEAREST)

img_array = np.array(img)
img_array = img_array[np.newaxis, :]
img_array = img_array.astype(np.float)
img_array = imagenet_utils.preprocess_input(img_array)

#Original image
plt.imshow(img_array[0])
plt.show()

In [0]:
# Get model
model = vgg16.VGG16()

# Strip softmax layer
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("deep_taylor", model)

# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(img_array)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))
# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))
plt.axis('off')
plt.savefig("readme_example_analysis.png")

deconvnet, guided_backprop, pattern, lrp